<a href="https://colab.research.google.com/github/dxmai/CS114.L21.KHCL/blob/main/FinalProject/FakeNews_Data_Include_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json

#Trang tin giả

##Activistpost

In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.activistpost.com/'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('h3.content-list-title')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('a.next::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)


    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018'] 
        available = False 
        date = response.css('.entry-meta span::text').get() 
        for year in yearsList: 
            if year in date: 
                available = True 
                break
        if available: 
            title = response.css('.entry-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                if count == 0:
                    count += 1
                    continue
                sentence = sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }

In [24]:
activist = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/activistpost.json')
activist.dropna(inplace=True)
activist

,date,title,text,is_fake
0,2021-05-28,US Mint Delays Silver Shipments Due To “Global...,Interest in silver is soaring (both for indust...,1
1,2021-06-28,"Research Paper Exposes Cybersecurity, Environm...",A 2018 survey revealed did NOT want to live i...,1
2,2021-05-29,DeSantis’s Anti-Riot Law Undermines Two Import...,When Florida Gov. Ron DeSantis spent the last ...,1
3,2021-05-28,Another Massive Cargo Ship Was Just Stuck In t...,To quote the great Los Angeles sportscaster Vi...,1
4,2021-06-25,BofA Crashes The “Transitory” Party: Sees Up T...,"At the start of May, when observing the avalan...",1
...,...,...,...,...
10270,2019-12-06,Comedian Fined $35k for Offensive Joke (And Ot...,Are you ready for this week’s absurdity? Here’...,1
10271,2019-12-06,Bill Gates Wants to Export India’s National ID...,It’s not just a social credit score system spr...,1
10272,2019-12-05,Pro Tip: Mentally Replace All Uses Of “Conspir...,The corrupt mechanisms which gave rise to the ...,1
10273,2019-12-05,"Despite Expert Warnings and Accidents, Elected...","According to many experts, Automated Vehicles ...",1


##Natural News

In [ ]:
import scrapy


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.naturalnews.com/all-posts.html'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('div.f-tabbed-list-content')
        allLinks = getLinkInDiv.css('h2 a::attr(href)').getall()

        for each_link in allLinks:
            each_link = 'https://www.naturalnews.com/' + each_link
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('.pagination-next a::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        date = response.css('.Article-Author::text').get()
        date = date.replace(' by: ', '')
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('.entry-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\t', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }

In [23]:
natural = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/naturalnews.json')
natural.dropna(inplace=True)
natural

,date,title,text,is_fake
0,2021-06-29,Honda announces plan to design and manufacture...,) Japanese automobile manufacturer Honda Motor...,1
1,2021-06-29,Russia calls America a ‘liberal totalitarian s...,) Russian authoritiesof the failing U.S. regim...,1
2,2021-06-29,"In Joe Biden’s America, Whites are the enemy",) Whatthose silly white people so nervous abou...,1
3,2021-06-29,Heart health and drug safety: Common cold medi...,) The common cold is caused by a viral infecti...,1
4,2021-06-29,Garbage-gobbling machine combats marine pollut...,) Afloating in the water has been cleaning up ...,1
...,...,...,...,...
11980,2020-03-03,NHS: Healthcare facilities across the UK will ...,) As the world scrambles to prepare and combat...,1
11981,2020-03-03,World Bank “pandemic bonds” may explain why th...,) Even as the first American dies of the Wuhan...,1
11982,2020-03-03,Estimated 300 – 500 coronavirus cases already ...,) One of the more fascinating things to note i...,1
11983,2020-03-04,One mistake is all it takes: 6 Dangerous survi...,) There are many beliefs about survival and pr...,1


##Zerohedge

In [ ]:
import scrapy
dateValid = True

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.zerohedge.com/'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('div.Article_mobileNonSticky__PmGNH')
        allLinks = getLinkInDiv.css('h2 a::attr(href)').getall()

        for each_link in allLinks:
            if dateValid:
                each_link = 'https://www.zerohedge.com' + each_link
                yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('a.SimplePaginator_next__15okP::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        global dateValid
        date = response.css('.ArticleFull_headerFooter__date__3T7FN::text').get()
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('.ArticleFull_title__2cUI6::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            paragraph = response.css('.NodeContent_body__2clki p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                sentence = sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }
        else:
            dateValid = False

In [22]:
zerohedge = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/zerohedge.json')
zerohedge.dropna(inplace = True)
zerohedge

,date,title,text,is_fake
0,2021-07-01 09:05:00,Complacent Goldilocks Got Eaten By Bear,"This morning I just watched a massive, fully l...",1
1,2021-07-01 09:28:00,New Video Shows Surfside Condo's Parking Garag...,Video recorded moments before the Champlain To...,1
2,2021-07-01 08:35:00,Almost 15 Million Americans Remain On Governme...,With more states ending their emergency handou...,1
3,2021-07-01 08:45:00,Nio Shares Pop 3% Pre-Market After Company Buc...,Shares of EV automaker NIO are up about 3% in ...,1
4,2021-07-01 07:15:00,Trump Organization CFO Alan Weisselberg Surren...,Weisselberg has pleaded not guilty to the char...,1
...,...,...,...,...
366,2021-06-30 20:05:00,Charges Filed Against Trump Org And CFO Alan W...,A Manhattan grand jury has filed the anticipat...,1
367,2021-07-01 12:26:00,Rescue Operations Halted At Collapsed Surfside...,One week after the Champlain Towers South buil...,1
368,2021-07-01 12:30:00,Ireland One Of 9 Holdouts Who Refused To Sign ...,"In a rare scoop, the Irish Times just reporte...",1
369,2021-07-01 06:44:00,Kamala Harris Staffers Are Leaking -- And Her ...,Vice President Kamala Harris' office is a toxi...,1


##Prntly

In [ ]:
import scrapy
dateValid = True

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://prntly.com/category/science/',
            'https://prntly.com/category/politics/',
            'https://prntly.com/category/world/europe/',
            'https://prntly.com/category/world/asia/',
            'https://prntly.com/category/world/middle-east/',
            'https://prntly.com/category/local-news/',
            'https://prntly.com/category/trade-jobs/',
            'https://prntly.com/category/immigration/'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('div.entry-content-holder')
        allLinks = getLinkInDiv.css('h2 a::attr(href)').getall()

        for each_link in allLinks:
            if dateValid:
                yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('a.next::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        global dateValid
        date = response.css('.entry-date::text').get()
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('h1.entry-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 4:
                    break
                sentence = sentence.replace('\xa0', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }
        else:
            dateValid = False


In [21]:
prntly = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/prntly.json')
prntly.dropna(inplace=True)
prntly

,date,title,text,is_fake
0,2020-01-28,Woman With C-Virus BRAGS Online About Sneaking...,A chinese national from the quarantined city o...,1
1,2020-02-07,Death of the first whistleblower of the Corona...,"The death of Dr. Li Wenliang has spark anger, ...",1
2,2020-01-27,Watch: Can China’s outbreak really be containe...,,1
3,2020-02-28,USA Leads The World In Coronavirus Recovery At...,"\nAcross the globe, the media spreads panic ab...",1
4,2020-02-28,BREAKING: Coronavirus Infected SPIKE In South ...,Seoul- over 2000 people have now come down wit...,1
...,...,...,...,...
568,2020-01-13,Dem attempting to insult Trump’s Farsi tweet m...,A democrat from Florida learned the hard way t...,1
569,2020-01-13,Breaking News: United Kingdom Claims Iran Brie...,In the midst of street protests on the verge o...,1
570,2020-01-13,president Trump’s Farsi tweet to iranian prote...,"President Trump made Iranian history, but not ...",1
571,2020-01-13,"As Iranians protest against the regime, Americ...",Confusion in the world of politics this week. ...,1


##Thegateway

In [ ]:
import scrapy
dateValid = True

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.thegatewaypundit.com/'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        if dateValid:
            getLinkInDiv = response.css('div.tgp-post')
            allLinks = getLinkInDiv.css('.entry-archive-title a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('.next a::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        global dateValid
        date = response.css('.entry-meta-text span::text')[2].get()
        date = date.replace('\nPublished ', '')
        temp = date.split(' at', 1)
        date = temp[0]
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('h1.entry-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }
        else:
            dateValid = False


In [20]:
thegateway = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/thegateway.json')
thegateway.dropna(inplace=True)
thegateway

,date,title,text,is_fake
0,"July 1, 2021",Man Tackles Woman to the Ground and Sexually A...,Violent crime and murder are skyrocketing in N...,1
1,"July 1, 2021",FIREWORKS: Radical Far-Left Media Voices From ...,Members of various media outlets attacked one ...,1
2,"July 1, 2021",Today the Chinese Communist Party Celebrates 1...,Check it out .Advertisement - story continues ...,1
3,"July 1, 2021",Discrimination Lawsuit Filed Against Chicago-A...,The Southwest Legal Foundation alleges violat...,1
4,"July 1, 2021",Hit Me Baby One More Time – Judge Shoots Down ...,Advertisement - story continues below:A judge ...,1
...,...,...,...,...
36658,"June 30, 2021",Report: Ashli Babbitt Shooter Is Member of Mik...,"Display at funeral service for Ashli Babbitt, ...",1
36659,"June 30, 2021","“If You Think I’m Kidding, I’m Not” – Joe Bide...",Joe Biden on Wednesday called Congresswoman Ma...,1
36660,"June 30, 2021",BREAKING: New York DOJ to Indict Trump CFO and...,President Trump speaks at Save America rally i...,1
36661,"June 30, 2021",“Yes” – Trump When Asked if He Has Made Up His...,President Trump sat down with Fox News host Se...,1


##Conservativedailypost 

In [ ]:
import scrapy
dateValid = True


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://conservativedailypost.com/category/politics/'
            'https://conservativedailypost.com/category/u-s/',
            'https://conservativedailypost.com/category/world/'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        if dateValid:
            getLinkInDiv = response.css('div.mob')
            allLinks = getLinkInDiv.css('h4 a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('a.next::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        global dateValid
        date = response.css('.date-published::text')[-1].get()
        date = date.replace('\n', '')
        date = date.replace('\t', '')
        date = date.replace('\r', '')
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('h1.min-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            paragraph = response.css('.content-container p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }
        else:
            dateValid = False

In [19]:
conservative = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/conservative.json')
conservative.dropna(inplace=True)
conservative

,date,title,text,is_fake
0,2021-07-01 13:00:00,"\rUSB Flash Drives Stolen, Transferred, Used I...",In multiple swing states flash drives (USBs) u...,1
1,2021-07-01 17:54:00,\rPA Supreme Court Overturns Cosby Conviction,The supreme court of Pennsylvania has found th...,1
2,2021-07-01 18:18:00,\rDelta Variant Being Used For Next Round Of L...,CNN rolled out Dr. Peter Hotez of Houston to s...,1
3,2021-07-01 19:01:00,\rNavy Forces Unit To March With ‘Gay’ US Flag...,Active duty members of the in San Diego were ...,1
4,2021-07-01 16:41:00,"\rFauci Announces Two Americas…. One Vax, One ...",In an appearance on Dom Lemon’s CNN panic hour...,1
...,...,...,...,...
5121,2018-01-23 00:29:00,\rChina Weighs In On “Superior” Action As Comm...,Just as the Soviet Union made during the previ...,1
5122,2018-01-23 00:23:00,\rArabs Ejected: Pence Rocks Chamber As “Your ...,U.S. Vice President Mike Pence told Israel’s K...,1
5123,2018-01-23 00:16:00,\r“Addicted To The Attention”: Serial Offender...,Judge William Raines is done feeling sorry. “I...,1
5124,2018-01-22 23:45:00,\rBank Calls 911 As Man Issues “Differing From...,Police report on an unusual situation involvin...,1


##Dailyheadlines

In [ ]:
import scrapy
dateValid = True


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'http://dailyheadlines.com/'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        if dateValid:
            getLinkInDiv = response.css('h2.entry-title')
            allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('.pagination a::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        global dateValid
        date = response.css('.post-meta-date::text')[-1].get()
        date = date.replace('\n', '')
        date = date.replace(' ' * 21, '')
        print(date)
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('h2.post-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            title = title.replace(' ' * 12, '')
            title = title.replace(' ' * 9, '')
            paragraph = response.css('.entry-content p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 1
            }
        else:
            dateValid = False



In [18]:
dailyheadline = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/dailyheadline.json')
dailyheadline.dropna(inplace=True)
dailyheadline

,date,title,text,is_fake
0,2021-06-25,73-Year-Old Pastor and Purple Heart Veteran Ar...,The witchhunt from the January 6 so-called “ri...,1
1,2021-06-25,Leftist Thug Reporters Threatening Anyone Asso...,“They’re coming for me because I fight for the...,1
2,2021-06-24,ALERT: Kamala Now Telling People To Harass The...,Biden’s July 4th vaccine goal is falling short...,1
3,2021-07-01,JAWDROPPING VIDEO: Trump Already Knows! He Alw...,It appears that the time for the BOOM to drop ...,1
4,2021-06-25,Biden Caught Openly Mocking Proud Gun Owners!,"During Joe Biden’s speech, he tried to hit at ...",1
...,...,...,...,...
2573,2021-04-08,"Alex Jones Saw Some Kids In Trouble, What He D...","In a now-viral video, conservative radio host ...",1
2574,2021-05-19,FRAUD Fauci Just Made An Absolutely SICKENING ...,Since Dr. Fauci has stepped onto the world sta...,1
2575,2021-06-24,"Biden Tweets, Then Mysteriously Deletes Haunti...",We all know that Joe Biden is not actually wri...,1
2576,2021-06-23,Trey Gowdy Has A Brutal Message For Democrats!,"In an interview with Fox News, Former Republic...",1


#Trang tin chính thống

##Csmonitor **Bị chặn tên miền rồiiiiiii** ;;-;;

In [ ]:
import scrapy
dateValid = True


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.csmonitor.com/World/Americas/(view)/all',
            'https://www.csmonitor.com/World/Making-a-difference/(view)/all',
            'https://www.csmonitor.com/World/Points-of-Progress/(view)/all',
            'https://www.csmonitor.com/World/(view)/all',
            'https://www.csmonitor.com/World/Africa/(view)/all',
            'https://www.csmonitor.com/World/Asia-Pacific/(view)/all',
            'https://www.csmonitor.com/World/Asia-South-Central/(view)/all',
            'https://www.csmonitor.com/World/Europe/(view)/all',
            'https://www.csmonitor.com/World/Middle-East/(view)/all',
            'https://www.csmonitor.com/USA/(view)/all',
            'https://www.csmonitor.com/USA/Politics/(view)/all',
            'https://www.csmonitor.com/USA/Politics/monitor_breakfast/(view)/all',
            'https://www.csmonitor.com/USA/Politics/Politics-Watch/(view)/all',
            'https://www.csmonitor.com/USA/Foreign-Policy/(view)/all',
            'https://www.csmonitor.com/USA/Military/(view)/all',
            'https://www.csmonitor.com/USA/Justice/(view)/all',
            'https://www.csmonitor.com/USA/Society/(view)/all',
            'https://www.csmonitor.com/Business/(view)/all',
            'https://www.csmonitor.com/Science/(view)/all',
            'https://www.csmonitor.com/Environment/(view)/all',
            'https://www.csmonitor.com/Technology/(view)/all',
            'https://www.csmonitor.com/The-Culture/(view)/all',
            'https://www.csmonitor.com/The-Culture/Arts/(view)/all',
            'https://www.csmonitor.com/The-Culture/Music/(view)/all',
            'https://www.csmonitor.com/The-Culture/Movies/(view)/all',
            'https://www.csmonitor.com/The-Culture/TV/(view)/all',
            'https://www.csmonitor.com/The-Culture/In-a-Word/(view)/all',
            'https://www.csmonitor.com/The-Culture/The-Home-Forum/(view)/all',
            'https://www.csmonitor.com/The-Culture/Food/(view)/all'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        if dateValid:
            getLinkInDiv = response.css('div.ezv-listing')
            allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            each_link = 'https://www.csmonitor.com' + each_link
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = 'https://www.csmonitor.com' + response.css('.ui-brdr-widget a::attr(href)')[-1].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        global dateValid

        date = response.css('.eza-publish_date::text').get()
        date = date.replace('\n', '')
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('h1.eza-title::text').get()
            title = title.replace('\n', '')
            title = title.replace('\t', '')
            paragraph = response.css('.story-two p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 0
            }
        else:
            dateValid = False

##Economist

In [ ]:
import scrapy


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://www.economist.com/leaders',
            'https://www.economist.com/briefing',
            'https://www.economist.com/united-states',
            'https://www.economist.com/asia',
            'https://www.economist.com/the-americas',
            'https://www.economist.com/china',
            'https://www.economist.com/middle-east-and-africa',
            'https://www.economist.com/europe',
            'https://www.economist.com/britain',
            'https://www.economist.com/international',
            'https://www.economist.com/business',
            'https://www.economist.com/finance-and-economics',
            'https://www.economist.com/science-and-technology',
            'https://www.economist.com/books-and-arts',
            'https://www.economist.com/graphic-detail',
            'https://www.economist.com/obituary'
        ]

        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('div.teaser__text')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            each_link = 'https://www.economist.com' + each_link
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('.ds-pagination__nav--next a::attr(href)')[0].get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        date = response.css('.article__dateline-datetime::text').get()
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('.article__headline::text').get()
            paragraph = response.css('.article__body-text::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 0
            }

In [17]:
eco = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/economist.json')
eco.dropna(inplace=True)
eco

,date,title,text,is_fake
0,2021-06-24,We’re hiring: a news assistant in Tokyo,"is seeking a . This is an exciting, multiface...",0
1,2021-06-24,An important census product may soon use synth...,"(), which is sent to around 1% of America’s p...",0
2,2021-06-24,Myanmar’s civil war is becoming bloodier and m...,"bicycle, wearing a -shirt emblazoned with a c...",0
3,2021-06-24,America’s Supreme Court is less one-sided than...,America’s Supreme Court seemed destined for a...,0
4,2021-06-12,The anti-graft unit of China’s Communist Party...,"June 1st Shi Zhaoqing, a local boss in China’...",0
...,...,...,...,...
12600,2020-08-20,"For your summer getaway, try an imaginary city","a summer getaway? Try the city of Isidora, “w...",0
12601,2019-01-17,Indonesia’s economic growth is being held back...,"in Singapore on January 15th, Indonesian offi...",0
12602,2018-09-20,The leaders of the two Koreas put on another g...,"THE setting keeps changing; the pictures, not ...",0
12603,2018-04-14,Kinder Morgan’s attempt to build a pipeline re...,"ALMOST all Canada’s oil and gas is landlocked,...",0


##APNews

In [ ]:
import scrapy


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://apnews.com/hub/coronavirus-pandemic',
            'https://apnews.com/hub/politics',
            'https://apnews.com/hub/sports',
            'https://apnews.com/hub/entertainment',
            'https://apnews.com/hub/lifestyle',
            'https://apnews.com/hub/oddities',
            'https://apnews.com/hub/travel',
            'https://apnews.com/hub/technology',
            'https://apnews.com/hub/ap-fact-check',
            'https://apnews.com/hub/business',
            'https://apnews.com/hub/us-news',
            'https://apnews.com/hub/health',
            'https://apnews.com/hub/science',
            'https://apnews.com/hub/world-news',
            'https://apnews.com/hub/religion'
        ]

        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        getLinkInDiv = response.css('div.CardHeadline')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            each_link = 'https://apnews.com' + each_link
            yield scrapy.Request(each_link, callback=self.parse_href)

        #next_page = response.css('.ds-pagination__nav--next a::attr(href)')[0].get()
        #if next_page is not None:
        #    next_page = response.urljoin(next_page)
        #    yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        date = response.css('.Timestamp::attr(title)').get()
        temp = date.split(' ', 1)
        date = temp[0]
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('.CardHeadline h1::text').get()
            paragraph = response.css('.Article p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 3:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 0
            }

In [ ]:
apnews = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/apnews.json')
apnews

,date,title,text,is_fake
0,2021-07-02,Dogs on display: Museum fetes 200 years of car...,"COLUMBUS, Ohio (AP) — In a 1970 Beetle Bailey ...",0
1,2021-07-03,"Ransomware hits hundreds of US companies, secu...",WASHINGTON (AP) — A ransomware attack paralyze...,0
2,2021-07-02,Paris airport workers block terminal to protes...,PARIS (AP) — Paris airport workers protesting ...,0
3,2021-07-02,‘Normal kid’ Grealish an England fan favorite ...,"BURTON-ON-TRENT, England (AP) — Fresh from an ...",0
4,2021-05-19,"UN urges more vaccines for Africa, with only 2...",UNITED NATIONS (AP) — The U.N. Security Counci...,0
...,...,...,...,...
731,2021-07-01,VIRUS DIARY: The unfinished business of a fune...,"NASHVILLE, Tenn. (AP) — It dawned on me recent...",0
732,2021-07-01,Whither #MeToo? Chilling effect of Cosby rever...,"When Indira Henard, director of the DC Rape Cr...",0
733,2021-07-02,AP Week in Pictures: North America,"JUNE 25 - JULY 1, 2021This photo gallery highl...",0
734,2021-07-03,Restaurateurs contend with new challenges for ...,"NEW KENSINGTON, Pa. (AP) — Restaurants were hi...",0


In [ ]:
apnews.dropna(inplace=True)
apnews

,date,title,text,is_fake
0,2021-07-02,Dogs on display: Museum fetes 200 years of car...,"COLUMBUS, Ohio (AP) — In a 1970 Beetle Bailey ...",0
1,2021-07-03,"Ransomware hits hundreds of US companies, secu...",WASHINGTON (AP) — A ransomware attack paralyze...,0
2,2021-07-02,Paris airport workers block terminal to protes...,PARIS (AP) — Paris airport workers protesting ...,0
3,2021-07-02,‘Normal kid’ Grealish an England fan favorite ...,"BURTON-ON-TRENT, England (AP) — Fresh from an ...",0
4,2021-05-19,"UN urges more vaccines for Africa, with only 2...",UNITED NATIONS (AP) — The U.N. Security Counci...,0
...,...,...,...,...
731,2021-07-01,VIRUS DIARY: The unfinished business of a fune...,"NASHVILLE, Tenn. (AP) — It dawned on me recent...",0
732,2021-07-01,Whither #MeToo? Chilling effect of Cosby rever...,"When Indira Henard, director of the DC Rape Cr...",0
733,2021-07-02,AP Week in Pictures: North America,"JUNE 25 - JULY 1, 2021This photo gallery highl...",0
734,2021-07-03,Restaurateurs contend with new challenges for ...,"NEW KENSINGTON, Pa. (AP) — Restaurants were hi...",0


##Npr (**load_more style**)

In [ ]:
import scrapy
offset, times = 0, 0
valid = True
res = ''
dictionary = {
    'national/': [1003, 25],
    'politics/': [1014, 25],
    'health/': [1128, 23],
    'technology/': [1019, 24],
    'world/': [1004, 22],
    'business/': [1006, 25],
    'science/': [1007, 24]
}


class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        global times, res, offset, valid
        urls = [
            'https://www.npr.org/sections/national/',
            'https://www.npr.org/sections/politics/',
            'https://npr.org/sections/health/'
            'https://npr.org/sections/technology/'
            'https://www.npr.org/sections/world/'
            'https://www.npr.org/sections/business/'
            'https://www.npr.org/sections/science/',
        ]

        for url in urls:
            for key in dictionary.keys():
                if url.endswith(key):
                    res = key
                    break
            times = 0
            offset = 0
            valid = True
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        global offset, times
        getLinkInDiv = response.css('h2.title')
        allLinks = getLinkInDiv.css('a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        if valid:
            next_page = 'https://www.npr.org/get/'
            temp = dictionary[res][0]
            next_page = next_page + str(temp)
            next_page = next_page + '/render/partial/next?start='
            if times == 0:
                offset += dictionary[res][1]
                times += 1
            else:
                offset += 24
            next_page = next_page + str(offset)
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        global valid
        yearsList = ['2021', '2020', '2019', '2018']
        available = False
        date = response.css('.date::text').get()
        for year in yearsList:
            if year in date:
                available = True
                break
        if available:
            title = response.css('.storytitle h1::text').get()
            paragraph = response.css('.storylocation p::text').getall()
            text = ''
            count = 0
            for sentence in paragraph:
                if count == 0 or count == 1 or count == 2:
                    count += 1
                    continue
                if count == 6:
                    break
                sentence = sentence.replace('\xa0', '')
                sentence = sentence.replace('\n', '')
                sentence = sentence.replace('\r', '')
                sentence = sentence.replace('\t', '')
                text = text + sentence
                count += 1
            yield {
                'date': date,
                'title': title,
                'text': text,
                'is_fake': 0
            }
        else:
            valid = False

In [10]:
npr_national = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/npr_national.json')
npr_business = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/npr_business.json')
npr_science = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/npr_science.json')
npr_health = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/npr_health.json')
npr_tech = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/npr_tech.json')
npr_world = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/npr_world.json')
npr_politic = pd.read_json('https://raw.githubusercontent.com/dxmai/CS114.L21.KHCL/main/FinalProject/Include_Text/npr_politics.json')

In [15]:
npr_list = [npr_science, npr_health, npr_tech, npr_world, npr_politic]
dataset = npr_national.append(npr_business, ignore_index=True)
for i in npr_list:
  dataset = dataset.append(i, ignore_index=True)
dataset.dropna(inplace=True)
dataset

,date,title,text,is_fake
0,"Updated July 2, 2021",Judge Approves Company's Withdrawal From Co-Ma...,A #FreeBritney activist protests against keepi...,0
1,"Updated July 2, 2021",Boy Scouts Of America Reaches Historic Settlem...,The Boy Scouts of America has reached a settle...,0
2,"Updated July 1, 2021","Trump's Family Business, CFO Weisselberg Are C...","Allen Weisselberg, the Trump Organization's lo...",0
3,"Updated July 1, 2021",The Justice Department Is Pausing Federal Exec...,Attorney General Merrick Garland ordered a pau...,0
4,"July 1, 2021",Judge Orders Release Of Wisconsin Woman In Sle...,"Anissa Weier, pictured in 2017, one of two Wis...",0
...,...,...,...,...
45299,"March 13, 2020","In Reversal, Trump Administration Now Urges Ag...",The White House is taking a different view on ...,0
45300,"March 13, 2020",Michael Tubbs: What Does It Take To Transform ...,Michael Tubbs is currently serving as the 82nd...,0
45301,"March 12, 2020",Brazilian Official Who Met Trump Last Weekend ...,President Trump and other U.S. officials met w...,0
45302,"March 13, 2020",President Trump Declares National Emergency As...,President Trump holds a news conference about ...,0


#Tổng kết số lượng

Tổng kết số lượng tìm được (đã loại bỏ những trường hợp có giá trị bị null):

Tin thật: 45290 + 730 + 12605 = 58625

Tin giả: 2578 + 5126 + 36663 + 573 + 371 + 11985 +  10275 = 67571